### 모델최적화 필요/ 또는 모델의 간소화(테스트 도중 너무 오래걸려서 실패)

In [ ]:
!pip install tqdm_joblib
!pip install numpy==1.26.4
!pip install pandas==2.2.2
!pip install scikit-learn==1.3.2
!pip install catboost==1.2.7

In [2]:
!pip install tqdm

You should consider upgrading via the 'C:\Users\tjddl\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from imblearn.under_sampling import RandomUnderSampler
from tqdm import tqdm

# 기본 변환 함수들
def convert_count_str(val):
    if pd.isna(val):
        return 0.0
    val = str(val).strip()
    if "회 이상" in val:
        return 6.0
    m = re.search(r'(\d+)', val)
    if m:
        return float(m.group(1))
    return 0.0

donor_age_mapping = {
    '만20세 이하': 3, '만21-25세': 5, '만26-30세': 4, '만31-35세': 2,
    '만36-40세': 1, '만41-45세': 0, '알 수 없음': 0
}

def convert_donor_age(val):
    if pd.isna(val):
        return np.nan
    return donor_age_mapping.get(str(val).strip(), np.nan)

def convert_nan_to_string(df, category_columns):
    df_copy = df.copy()
    for col in category_columns:
        if isinstance(df_copy[col].dtype, pd.CategoricalDtype):
            categories = df_copy[col].cat.categories.tolist()
            df_copy[col] = df_copy[col].astype(str).replace('nan', 'Unknown')
            if 'Unknown' not in categories:
                categories.append('Unknown')
            df_copy[col] = pd.Categorical(df_copy[col], categories=categories)
        else:
            df_copy[col] = df_copy[col].fillna('Unknown')
    return df_copy

def get_categorical_feature_indices(df):
    cat_features = []
    for idx, (column, dtype) in enumerate(df.dtypes.items()):
        if dtype == 'category':
            cat_features.append(idx)
    return cat_features

def load_base_weights():
    """시술별 기본 가중치 정의"""
    weights = pd.DataFrame({
        'IVF': [  # IVF는 배아 품질과 수가 매우 중요
            4.8, 4.8, 5.0, 4.7, 4.7,  # 배아/배란 관련
            2.8, 2.8, 3.2, 3.2, 2.8, 2.8, 2.5,  # 불임 원인
            3.8, 3.5, 3.8, 3.5, 2.8, 3.2, 2.8, 2.8, 2.8, 2.8,  # 세부 불임 원인
            4.8, 4.5, 4.5, 5.0, 0.0, 4.2, 4.8, 0.0, 4.2, 4.8, 0.0,  # 시술/임신/출산 횟수
            4.8, 4.2, 4.2, 4.8, 4.2, 4.8, 4.2, 4.5, 4.5,  # 배아/난자 수
            3.2, 3.2, 2.8, 2.8, 2.5,  # 난자 처리
            3.8, 3.8, 2.8, 2.2,  # 출처/기증자
            4.8, 4.8, 3.5, 2.8,  # 배아 사용
            4.8, 4.5, 4.5, 4.8, 4.8  # 경과일
        ],
        'ICSI': [  # ICSI는 미세수정과 정자 품질이 매우 중요
            4.5, 4.5, 4.8, 4.8, 4.8,
            3.2, 3.2, 3.0, 3.0, 2.8, 2.8, 2.5,
            3.5, 4.8, 3.8, 3.5, 2.8, 3.0, 4.8, 4.8, 4.8, 4.8,
            4.8, 4.5, 4.5, 4.8, 0.0, 4.2, 4.8, 0.0, 4.2, 4.8, 0.0,
            4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.5, 4.5,
            3.8, 3.8, 3.2, 3.5, 3.0,
            3.8, 4.5, 2.8, 2.5,
            4.8, 4.8, 3.5, 2.8,
            4.8, 4.5, 4.5, 4.8, 4.8
        ],
        'IUI': [  # IUI는 배란 시기와 정자 운동성이 매우 중요
            4.8, 4.8, 2.8, 2.5, 2.5,
            3.8, 3.8, 3.8, 3.8, 3.8, 3.8, 3.5,
            3.2, 4.8, 4.8, 3.8, 2.8, 2.8, 4.8, 4.8, 4.8, 4.8,
            3.8, 3.8, 3.8, 0.0, 4.8, 3.8, 0.0, 4.8, 3.8, 0.0, 4.8,
            2.8, 2.8, 2.8, 2.8, 2.8, 2.8, 2.8, 2.8, 2.8,
            2.8, 2.8, 2.8, 2.8, 2.8,
            3.2, 4.8, 2.8, 3.8,
            3.8, 3.8, 3.8, 2.8,
            4.2, 3.8, 3.8, 3.8, 3.8
        ],
        'FER': [  # 동결배아 이식은 해동과 배아 생존이 매우 중요
            4.2, 4.2, 4.8, 4.5, 4.5,
            2.8, 2.8, 2.8, 2.8, 2.8, 2.8, 2.5,
            3.2, 3.2, 3.5, 3.2, 2.8, 3.0, 2.8, 2.8, 2.8, 2.8,
            4.5, 4.2, 4.2, 4.8, 0.0, 4.0, 4.5, 0.0, 4.0, 4.5, 0.0,
            4.5, 4.0, 4.0, 4.5, 4.0, 4.8, 4.0, 4.8, 4.8,
            3.5, 3.5, 3.2, 3.2, 2.8,
            3.5, 3.5, 2.8, 2.2,
            4.8, 3.8, 3.2, 2.8,
            4.5, 4.8, 4.2, 4.8, 4.8
        ],
        'Generic D IVI': [  # 기증자 시술은 기증자 특성이 매우 중요
            4.2, 4.2, 4.5, 4.2, 4.2,
            2.5, 2.5, 2.8, 2.8, 2.5, 2.5, 2.2,
            3.2, 3.2, 3.5, 3.2, 2.5, 2.8, 3.8, 3.8, 3.8, 3.8,
            4.2, 4.0, 4.0, 0.0, 4.8, 3.8, 0.0, 4.8, 3.8, 0.0, 4.8,
            4.2, 3.8, 3.8, 4.2, 3.8, 4.2, 3.8, 4.0, 4.0,
            3.8, 3.8, 3.5, 3.5, 3.2,
            4.8, 4.8, 4.8, 4.8,
            4.5, 4.2, 4.8, 3.5,
            4.2, 4.0, 4.0, 4.2, 4.2
        ]
    })
    return weights
def load_model_specific_weights():
    """모델별 가중치 정의"""
    base_weights = load_base_weights()
    
    model_weights = {
        'xgb': {
            'IVF': base_weights['IVF'] * 1.1,
            'ICSI': base_weights['ICSI'] * 1.1,
            'IUI': base_weights['IUI'] * 0.9,
            'FER': base_weights['FER'] * 1.0,
            'Generic D IVI': base_weights['Generic D IVI'] * 1.0
        },
        'lgbm': {
            'IVF': base_weights['IVF'] * 1.0,
            'ICSI': base_weights['ICSI'] * 1.2,
            'IUI': base_weights['IUI'] * 1.1,
            'FER': base_weights['FER'] * 1.0,
            'Generic D IVI': base_weights['Generic D IVI'] * 1.1
        },
        'cat': {
            'IVF': base_weights['IVF'] * 1.2,
            'ICSI': base_weights['ICSI'] * 1.0,
            'IUI': base_weights['IUI'] * 1.0,
            'FER': base_weights['FER'] * 1.1,
            'Generic D IVI': base_weights['Generic D IVI'] * 1.0
        }
    }
    return model_weights

def apply_model_specific_weights(X, model_weights, model_name):
    """특정 모델에 대한 가중치 적용"""
    X_weighted = X.copy()
    weights = model_weights[model_name]
    
    batch_size = 10000
    total_rows = len(X_weighted)
    
    for start_idx in tqdm(range(0, total_rows, batch_size), 
                         desc=f"{model_name} 가중치 적용"):
        end_idx = min(start_idx + batch_size, total_rows)
        batch = X_weighted.iloc[start_idx:end_idx]
        
        for idx, row in batch.iterrows():
            procedure_type = row['시술 유형']
            if pd.isna(procedure_type):
                continue
            
            if procedure_type in weights:
                current_weights = weights[procedure_type]
            else:
                current_weights = weights['IVF']
                
            for feature in batch.columns:
                if feature in current_weights.index:
                    X_weighted.at[idx, feature] *= current_weights[feature]
    
    return X_weighted

# 메인 코드
print("데이터 로딩 중...")
train = pd.read_csv('train.csv').drop(columns=['ID'])
test = pd.read_csv('test.csv').drop(columns=['ID'])

print("학습 데이터 전처리 중...")
# '시술 당시 나이' 결측치 여부 추가
train['시술 당시 나이_missing'] = train['시술 당시 나이'].apply(lambda x: 1.0 if str(x).strip() == '알 수 없음' else 0.0)
test['시술 당시 나이_missing'] = test['시술 당시 나이'].apply(lambda x: 1.0 if str(x).strip() == '알 수 없음' else 0.0)

# '시술 당시 나이' 변환
age_mapping = {
    '만18-34세': 5, '만35-37세': 4, '만38-39세': 3, '만40-42세': 2,
    '만43-44세': 1, '만45-50세': 0, '알 수 없음': np.nan
}
train['시술 당시 나이'] = train['시술 당시 나이'].apply(lambda x: float(age_mapping.get(str(x).strip(), 0)))
test['시술 당시 나이'] = test['시술 당시 나이'].apply(lambda x: float(age_mapping.get(str(x).strip(), 0)))

# 횟수 관련 컬럼 변환
count_columns = ["총 임신 횟수", "총 출산 횟수", "총 시술 횟수", "IVF 시술 횟수", "DI 시술 횟수", "클리닉 내 총 시술 횟수"]
for col in tqdm(count_columns, desc="횟수 관련 컬럼 변환"):
    train[col] = train[col].astype(str).apply(convert_count_str)
    test[col] = test[col].astype(str).apply(convert_count_str)

print("나이 관련 데이터 변환 중...")
# 난자/정자 기증자 나이 변환
train['난자 기증자 나이'] = train['난자 기증자 나이'].astype(str).apply(convert_donor_age)
test['난자 기증자 나이'] = test['난자 기증자 나이'].astype(str).apply(convert_donor_age)
train['정자 기증자 나이'] = train['정자 기증자 나이'].astype(str).apply(convert_donor_age)
test['정자 기증자 나이'] = test['정자 기증자 나이'].astype(str).apply(convert_donor_age)

print("카테고리 데이터 변환 중...")
# 카테고리형 변수 처리
category_columns = [
    "시술 시기 코드", "시술 유형", "특정 시술 유형", "배란 유도 유형",
    "배아 생성 주요 이유", "IVF 임신 횟수", "DI 임신 횟수",
    "IVF 출산 횟수", "DI 출산 횟수", "난자 출처", "정자 출처"
]

X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

# 모델별 가중치 데이터 로드
model_weights = load_model_specific_weights()

# 데이터 불균형 처리
undersample = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
X_resampled, y_resampled = undersample.fit_resample(X, y)

# NaN을 문자열로 변환하고 카테고리형으로 변환
X_resampled = convert_nan_to_string(X_resampled, category_columns)
test = convert_nan_to_string(test, category_columns)

for col in category_columns:
    X_resampled[col] = X_resampled[col].astype("category")
    test[col] = test[col].astype("category")

# 학습/검증 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)

# 모델 설정
estimators = [
    ('xgb', Pipeline([
        ('weight', FunctionTransformer(
            lambda X: apply_model_specific_weights(X, model_weights, 'xgb')
        )),
        ('model', XGBClassifier(
            tree_method='gpu_hist',
            enable_categorical=True,
            n_estimators=150,
            max_depth=6,
            learning_rate=0.05
        ))
    ])),
    ('lgbm', Pipeline([
        ('weight', FunctionTransformer(
            lambda X: apply_model_specific_weights(X, model_weights, 'lgbm')
        )),
        ('model', LGBMClassifier(
            n_jobs=-2,
            n_estimators=150,
            learning_rate=0.05,
            categorical_feature=category_columns
        ))
    ])),
    ('cat', Pipeline([
        ('weight', FunctionTransformer(
            lambda X: apply_model_specific_weights(X, model_weights, 'cat')
        )),
        ('model', CatBoostClassifier(
            task_type='GPU',
            iterations=150,
            learning_rate=0.05,
            cat_features=get_categorical_feature_indices(X_train)
        ))
    ]))
]

stack_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=Pipeline([
        ('scaler', StandardScaler()),
        ('lr', LogisticRegression(max_iter=2000, n_jobs=1))
    ]),
    cv=5,
    n_jobs=1
)

# 그리드 서치
param_grid = {
    'final_estimator__lr__C': [0.1, 1.0, 10.0]
}

grid_search = GridSearchCV(
    stack_clf,
    param_grid,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    verbose=2
)

print("모델 학습 시작...")
grid_search.fit(X_train, y_train)

print("\n검증 성능 평가...")
y_val_pred = grid_search.best_estimator_.predict_proba(X_val)[:, 1]
val_auc = roc_auc_score(y_val, y_val_pred)
print(f"Validation ROC AUC: {val_auc:.5f}")

print("\n개별 모델 성능:")
for name, estimator in grid_search.best_estimator_.named_estimators_.items():
    y_val_pred_single = estimator.predict_proba(X_val)[:, 1]
    single_auc = roc_auc_score(y_val, y_val_pred_single)
    print(f"{name}: {single_auc:.5f}")

# 최종 예측 및 제출 파일 생성
print("\n최종 예측 생성 중...")
pred_proba = grid_search.best_estimator_.predict_proba(test)[:, 1]
submission = pd.DataFrame({
    'ID': [f"TEST_{i:05d}" for i in range(len(test))],
    'probability': pred_proba
})
submission.to_csv('model_specific_weights_submission.csv', index=False)
print("제출 파일 생성 완료")

데이터 로딩 중...
학습 데이터 전처리 중...


횟수 관련 컬럼 변환: 100%|██████████| 6/6 [00:02<00:00,  2.36it/s]


나이 관련 데이터 변환 중...
카테고리 데이터 변환 중...
모델 학습 시작...
Fitting 3 folds for each of 3 candidates, totalling 9 fits
